# Откройте Anaconda Prompt и выполните следующию команду (команду также можно выполнить из ячейки Jupyter)
# _pip install psycopg2_
# (для MacOS) 
# _pip install psycopg2-binary_

In [2]:
# Создаем соединение c базой данных (ВНИМАНИЕ! Ваши данные для подключения с БД могут отличаться от данного примера)
import psycopg2 # драйвер для postgresql
import getpass  # прячем параль, чтобы не писать его в параметрах

conn=psycopg2.connect(user='postgres',
                      database='test',
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password: ')
                        )
print(conn.get_dsn_parameters())

Insert password: ········
{'user': 'postgres', 'dbname': 'test', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [3]:
# Получение данных из БД с помощью pandas
import pandas as pd
query="SELECT * FROM order_log WHERE \"TIME\"=100000000 AND \"SECCODE\"='SBER';"
# Обратите внимание, что специальные символы Python в текстовых переменных следует отделять символом \,
# чтобы те воспринимались как обычный текст.

# Можно также читать запрос из текстового файла на диске
# with open('path\\query_test.sql','r') as query:
#     data=pd.read_sql_query(query.read(),conn)
# data.head(10)

In [17]:
# Получение данных из БД с помощью курсора
# Создаем курсор (атрибут созданного соединения)
cur=conn.cursor()

# Исполняем запрос командой execute
cur.execute("SELECT * FROM order_log WHERE \"TIME\"=100000000 AND \"SECCODE\"='SBER';")
# Запрос исполняется в БД и ждет затягивания в Python

# С помощью курсора можно узнать информацию о результатах запроса
print(cur.rowcount)    # возвращает количество записей (строк) в результате запроса
print(cur.description) # возвращает названия полей и их форматы

# Закачка данных осуществляется с помощью комад (fetchone, fetchmany(rowsnum), fethcall) 
data=cur.fetchall()
# rowsnum=10
# data=cur.fetchmany(rowsnum)
# Курсор позволяет закачивать данные частями, т.к. после очередной fethc команды курсор меняет свое  положение 

# Положения курсора можно регулировать атрибутом scroll 
cur.scroll(0,mode='absolute') # возвращает курсор на первую строку запроса
cur.scroll(1,mode='relative') # перемещае курсов на одну стороку вниз от текущего положения
dt

1550
(Column(name='NO', type_code=23), Column(name='SECCODE', type_code=25), Column(name='BUYSELL', type_code=18), Column(name='TIME', type_code=20), Column(name='ORDERNO', type_code=23), Column(name='ACTION', type_code=23), Column(name='PRICE', type_code=700), Column(name='VOLUME', type_code=20), Column(name='TRADENO', type_code=20), Column(name='TRADEPRICE', type_code=700))


AttributeError: module 'pandas' has no attribute 'shape'

In [10]:
# Курсор передает данные из запроса в виде списка кортежей. Чтобы получить DataFrame списко нужно преобразовать  
# и задать в нем названия полей

data1=pd.DataFrame(data,columns=[x.name for x in cur.description])
data1

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,6,SBER,S,100000000,6,1,109.99,3000,None,None
1,8,SBER,S,100000000,8,1,108.87,300,None,None
2,9,SBER,S,100000000,9,1,109.87,400,None,None
3,10,SBER,S,100000000,10,1,110.87,500,None,None
4,11,SBER,S,100000000,11,1,108.57,30,None,None
5,12,SBER,S,100000000,12,1,108.67,30,None,None
6,13,SBER,S,100000000,13,1,108.77,30,None,None
7,14,SBER,S,100000000,14,1,108.97,30,None,None
8,17,SBER,S,100000000,17,1,109.07,40,None,None
9,18,SBER,S,100000000,18,1,109.17,40,None,None


In [ ]:
# Немного тренировки с возможностями pandas (как-то маловато успели)

data1.pivot_table(index='PRICE',columns='BUYSELL',values=data.columns[7],aggfunc='sum')